![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### Portfolio uchun vazifa: Toshkent shahrida uylarning narxini aniqlash.

Ushbu amaliyotda sizning vazifangiz berilgan ma`lumotlar asosida Toshkent shahridagi uylarning narxini aniqlash.

In [318]:
#Importing and loading DataFrame
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score


RF_model = RandomForestRegressor()
Tree_model = DecisionTreeRegressor()
LR_model = LinearRegression()
ordinal_encoder = OrdinalEncoder()


df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv')
df.head()

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


# Ustunlar ta'rifi
- `location` - sotilayotgan uy manzili
- `district` - uy joylashgan tuman
- `rooms` - xonalar soni
- `size` - uy maydoni (kv.m)
- `level` - uy joylashgan qavat
- `max_levels` - ja'mi qavatlar soni
- `price` - uy narxi

## Vazifani CRSIP-DM Metolodgiyasi yordamida bajaring.
<img src="https://i.imgur.com/dzZnnYi.png" alt="CRISP-DM" width="800"/>

In [280]:
print(df.info(),'\n\n')
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7565 entries, 0 to 7564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   location    7565 non-null   object
 1   district    7565 non-null   object
 2   rooms       7565 non-null   int64 
 3   size        7565 non-null   object
 4   level       7565 non-null   int64 
 5   max_levels  7565 non-null   int64 
 6   price       7565 non-null   object
dtypes: int64(3), object(4)
memory usage: 413.8+ KB
None 




,rooms,level,max_levels
count,7565.000000,7565.000000,7565.000000
mean,2.625644,3.699273,6.038202
std,1.085201,2.237275,2.613271
min,1.000000,1.000000,1.000000
25%,2.000000,2.000000,4.000000
50%,3.000000,3.000000,5.000000
75%,3.000000,5.000000,9.000000
max,10.000000,19.000000,25.000000


##Preparing DF to ML

In [281]:
#df['size']=df['size'].astype(float)

In [282]:
df[df['size']=='Площадьземли:1сот']

,location,district,rooms,size,level,max_levels,price
5347,"город Ташкент, Яшнободский район, Дархон",Яшнободский,4,Площадьземли:1сот,3,5,150000


In [283]:
df=df[df['size']!='Площадьземли:1сот']

In [284]:
df['size']=df['size'].astype(np.float64)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [285]:
#df['price'] = df['price'].astype(int)

In [286]:
df[df['price']=='Договорная']

,location,district,rooms,size,level,max_levels,price
202,"город Ташкент, Яккасарайский район, Баходыра",Яккасарайский,3,119.0,3,9,Договорная
411,"город Ташкент, Яккасарайский район, Баходыра",Яккасарайский,4,160.0,4,9,Договорная
439,"город Ташкент, Мирзо-Улугбекский район, улица ...",Мирзо-Улугбекский,3,105.0,5,6,Договорная
460,"город Ташкент, Чиланзарский район, Чиланзар 1-...",Чиланзарский,3,90.0,6,8,Договорная
507,"город Ташкент, Яшнободский район, 1-й проезд А...",Яшнободский,2,48.0,4,4,Договорная
...,...,...,...,...,...,...,...
7039,"город Ташкент, Яшнободский район, Городок Авиа...",Яшнободский,1,38.7,3,8,Договорная
7196,"город Ташкент, Чиланзарский район, Чиланзар-16",Чиланзарский,2,51.0,3,4,Договорная
7323,"город Ташкент, Мирзо-Улугбекский район, жилой ...",Мирзо-Улугбекский,6,208.0,1,7,Договорная
7403,"город Ташкент, Учтепинский район, Чиланзар 14-...",Учтепинский,2,35.0,2,9,Договорная


In [287]:
df['price'] = df['price'].replace('Договорная', 0)
df['price'] = df['price'].astype(np.int64)

In [288]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7564 entries, 0 to 7564
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    7564 non-null   object 
 1   district    7564 non-null   object 
 2   rooms       7564 non-null   int64  
 3   size        7564 non-null   float64
 4   level       7564 non-null   int64  
 5   max_levels  7564 non-null   int64  
 6   price       7564 non-null   int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 472.8+ KB


In [289]:
df['size'] = df['size'].astype(int)

In [290]:
df['size'].replace(0,df.size.mean(),inplace=True)

In [291]:
df

,location,district,rooms,size,level,max_levels,price
0,"город Ташкент, Юнусабадский район, Юнусабад 8-...",Юнусабадский,3,57,4,4,52000
1,"город Ташкент, Яккасарайский район, 1-й тупик ...",Яккасарайский,2,52,4,5,56000
2,"город Ташкент, Чиланзарский район, Чиланзар 2-...",Чиланзарский,2,42,4,4,37000
3,"город Ташкент, Чиланзарский район, Чиланзар 9-...",Чиланзарский,3,65,1,4,49500
4,"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000
...,...,...,...,...,...,...,...
7560,"город Ташкент, Яшнободский район, Городок Авиа...",Яшнободский,1,38,5,5,24500
7561,"город Ташкент, Яшнободский район, 1-й проезд А...",Яшнободский,2,49,1,4,32000
7562,"город Ташкент, Шайхантахурский район, Зульфиях...",Шайхантахурский,2,64,3,9,40000
7563,"город Ташкент, Мирзо-Улугбекский район, Буюк И...",Мирзо-Улугбекский,1,18,1,4,11000


##Starting ML

###Linear Regression

In [292]:
train_set,test_set = train_test_split(df,test_size=0.2,random_state=42)
X_train=train_set.drop('price',axis=1)
y=train_set['price'].copy()

In [293]:
X_num = X_train.drop(['district','location'],axis=1)

In [294]:
num_pipeline = Pipeline([
          ('std_scaler', StandardScaler())
])

In [295]:
num_attribs = list(X_num)
cat_attribs = ['district','location']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OrdinalEncoder(), cat_attribs)
])

In [296]:
X_cat = X_train[['location']]
X_cat_dis = X_train[['district']]

In [297]:
X_prepared = full_pipeline.fit_transform(X_train)

In [298]:
X_prepared[0:5,:]

array([[ 3.43739212e-01, -2.47686724e-02, -7.57570840e-01,
         1.12775328e+00,  7.00000000e+00,  8.86000000e+02],
       [ 1.26367109e+00, -1.18041483e-02,  1.02576359e+00,
         1.12775328e+00,  1.10000000e+01,  1.36800000e+03],
       [-5.76192663e-01, -4.30715300e-02,  1.34096375e-01,
        -7.84169590e-01,  2.00000000e+00,  2.38000000e+02],
       [ 3.43739212e-01, -2.47686724e-02,  1.91743081e+00,
         1.12775328e+00,  5.00000000e+00,  6.27000000e+02],
       [-5.76192663e-01, -4.15462919e-02, -1.20340445e+00,
        -7.84169590e-01,  8.00000000e+00,  1.09100000e+03]])

In [299]:
LR_model.fit(X_prepared, y)

LinearRegression()

In [300]:
test_data = X_train.sample(5)
test_data

,location,district,rooms,size,level,max_levels
5094,"город Ташкент, Мирзо-Улугбекский район, Мирзо-...",Мирзо-Улугбекский,3,65,4,5
1369,"город Ташкент, Чиланзарский район, Шарк",Чиланзарский,3,92,7,9
5917,"город Ташкент, Шайхантахурский район, Массив б...",Шайхантахурский,4,84,4,4
6323,"город Ташкент, Чиланзарский район, Чиланзар 7-...",Чиланзарский,1,30,2,4
2531,"город Ташкент, Шайхантахурский район, Алишера ...",Шайхантахурский,3,82,3,3


In [301]:
test_label = y.loc[test_data.index]
test_label

5094    40000
1369    52500
5917    79000
6323    26500
2531    67000
Name: price, dtype: int64

In [302]:
test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared

array([[ 3.43739212e-01, -3.01070059e-02,  1.34096375e-01,
        -4.01785015e-01,  2.00000000e+00,  2.82000000e+02],
       [ 3.43739212e-01, -9.51629114e-03,  1.47159720e+00,
         1.12775328e+00,  6.00000000e+00,  8.37000000e+02],
       [ 1.26367109e+00, -1.56172437e-02,  1.34096375e-01,
        -7.84169590e-01,  7.00000000e+00,  9.05000000e+02],
       [-1.49612454e+00, -5.67986732e-02, -7.57570840e-01,
        -7.84169590e-01,  6.00000000e+00,  7.82000000e+02],
       [ 3.43739212e-01, -1.71424818e-02, -3.11737233e-01,
        -1.16655417e+00,  7.00000000e+00,  8.66000000e+02]])

In [303]:
predicted_data = LR_model.predict(test_data_prepared)
predicted_data

array([68277.60767946, 72716.4517355 , 87145.33483765, 23582.46329554,
       64566.50193424])

In [304]:
pd.DataFrame({'Prognoz':predicted_data, 'Real baxosi': test_label})

,Prognoz,Real baxosi
5094,68277.607679,40000
1369,72716.451735,52500
5917,87145.334838,79000
6323,23582.463296,26500
2531,64566.501934,67000


In [305]:
X_test = test_set.drop('price', axis=1)
X_test

,location,district,rooms,size,level,max_levels
1718,"город Ташкент, Яккасарайский район, 7-й проезд...",Яккасарайский,2,82,8,8
4715,"город Ташкент, Юнусабадский район, 2-й проезд ...",Юнусабадский,3,100,1,4
5022,"город Ташкент, Яшнободский район, жилой компле...",Яшнободский,4,120,2,4
1046,"город Ташкент, Мирабадский район, Нукус",Мирабадский,2,65,5,9
2707,"город Ташкент, Чиланзарский район, Чиланзар кв...",Чиланзарский,3,75,9,9
...,...,...,...,...,...,...
333,"город Ташкент, Чиланзарский район, Чиланзар 7-...",Чиланзарский,2,58,1,9
5037,"город Ташкент, Учтепинский район, Кухна-Чупоно...",Учтепинский,2,44,4,5
3047,"город Ташкент, Мирзо-Улугбекский район, ц-1 Бу...",Мирзо-Улугбекский,3,78,3,4
4679,"город Ташкент, Шайхантахурский район, 1-я Ипакчи",Шайхантахурский,2,65,3,4


In [306]:
y_test = test_set['price'].copy()
y_test

1718     69900
4715     68000
5022    110000
1046     70000
2707     55000
         ...  
333      40000
5037     36000
3047    120000
4679     36000
4907     31000
Name: price, Length: 1513, dtype: int64

In [307]:
X_test_prepared = full_pipeline.fit_transform(X_test)

In [308]:
y_predicted = LR_model.predict(X_test_prepared)

In [309]:
y_predicted

array([ 66331.57138591,  90046.63892151, 116198.6117757 , ...,
        71187.70597155,  60660.54545266,  54687.26836341])

In [310]:
linear_mse = mean_squared_error(y_test, y_predicted)
linear_rmse = np.sqrt(linear_mse)
print(linear_rmse)

1342187.190941405


###Decision Tree Regressor

In [311]:
Tree_model.fit(X_prepared, y)

DecisionTreeRegressor()

In [312]:
y_predicted = Tree_model.predict(X_test_prepared)

In [313]:
linear_mse = mean_squared_error(y_test, y_predicted)
linear_rmse = np.sqrt(linear_mse)
print(linear_rmse)

1349431.8816841282


###Random Forest Regression

In [314]:
RF_model.fit(X_prepared, y)
y_predicted = RF_model.predict(X_test_prepared)

In [315]:
lin_mse = mean_squared_error(y_test, y_predicted)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

1345586.4305143396


###Cross Validation

In [316]:
X = df.drop("price", axis=1)
y = df["price"].copy()

X_prepared2 = full_pipeline.fit_transform(X)

In [317]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Std.dev:", scores.std())

In [321]:
scores = cross_val_score(LR_model, X_prepared2, y, scoring="neg_mean_squared_error", cv=5)
LR_rmse_scores = np.sqrt(-scores)

In [322]:
display_scores(LR_rmse_scores)

Scores: [  57645.04139042  105390.23640317   47729.80654172 1334992.40971328
  470442.64050178]
Mean: 403240.02691007365
Std.dev: 491368.8233172115


In [324]:
scores = cross_val_score(Tree_model, X_prepared2, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [  63689.01054708 1889233.11286427  244094.96615135   61385.72680161
 1922560.45803934   62376.10381789   47644.06606567 1889232.92331152
  288216.12842109  662283.44803096]
Mean: 713071.5944050795
Std.dev: 796566.7585460669


In [326]:
scores = cross_val_score(RF_model, X_prepared2, y, scoring="neg_mean_squared_error", cv=10)
LR_rmse_scores = np.sqrt(-scores)
display_scores(LR_rmse_scores)

Scores: [  65314.2639335   381380.56631028  206449.1305799    72345.8689371
  974082.86278154   90757.46557499  333988.84149085 1889382.72655058
  188484.1032633   675771.32256674]
Mean: 487795.71519887866
Std.dev: 543020.1444122399
